# Keras metrics

Instead of appending loss values to a list or a numpy array, we usually rely on something more convenient: tf.keras.metrics objects.

Keras comes with useful objects that support tracking a variable, such as the loss over an entire epoch, computing the average over all losses efficiently while allowing for graph mode train and test step functions/methods.


Let's assume we have one **epoch** with a small dataset that fills only **4 batches**, so to compute the average loss for this epoch, we want to average over four loss values. 

We can do this with a tf.keras.metrics.Mean object, which has an **update_state** method, that takes a scalar value to take into account for the running average, a **result** method, to obtain the result, and a **reset_states** method that resets the metric (after an epoch and between the training and validation steps).

To see what is going on we print the metric's result after each batch

In [26]:
import tensorflow as tf

loss_function = tf.keras.losses.MeanSquaredError()

# instantiate metric object (usually in the model's constructor, i.e. __init__, method)
loss_metric = tf.keras.metrics.Mean(name="loss")

# ITERATING OVER A NUMBER OF EPOCHS:
for e in range(4):

    # ITERATING OVER TRAINING DATA
    for batch in range(1000):

        model_output = tf.random.uniform(shape=(4,1))*10
        target = tf.random.uniform(shape=(4,1))*10

        loss = loss_function(target, model_output)

        loss_metric.update_state(values=loss)

    tf.print(f"Epoch {e}: loss: {loss_metric.result()}")

    # RESETTING THE METRICS
    loss_metric.reset_states()

    # ITERATING OVER VALIDATION DATA
    for batch in range(200):

        model_output = tf.random.uniform(shape=(4,1))*10

        target = tf.random.uniform(shape=(4,1))*10

        val_loss = loss_function(target, model_output)

        loss_metric.update_state(values=val_loss)

    tf.print(f"Epoch {e}: val_loss: {loss_metric.result()} \n")

    # RESETTING THE METRIC BEFORE NEXT EPOCH
    loss_metric.reset_states()

Epoch 0: loss: 15.873384475708008
Epoch 0: val_loss: 16.585254669189453 

Epoch 1: loss: 16.326271057128906
Epoch 1: val_loss: 17.245628356933594 

Epoch 2: loss: 16.922609329223633
Epoch 2: val_loss: 16.337326049804688 

Epoch 3: loss: 16.261056900024414
Epoch 3: val_loss: 16.288795471191406 



## Accuracy metrics for binary classification
As you see we can use tf.keras.metrics.Mean objects to compute running averages without using numpy or list appends. Keras comes with such metric objects for a number of different metrics that we might use to evaluate our model's performance, such as **BinaryAccuracy** in the context of binary classification and **CategoricalAccuracy** **TopKCategoricalAccuracy** in the context of multi-class classification. 

In [27]:
# instantiate metric object (usually in the model's constructor, i.e. __init__, method)
loss_metric = tf.keras.metrics.Mean(name="loss")
binary_accuracy_metric = tf.keras.metrics.BinaryAccuracy(name="accuracy")
loss_function = tf.keras.losses.BinaryCrossentropy()

# What happens in one epoch in the training loop:

# training on train data (4 batches)

for e in range(5):
    for train_batch in range(100):
        target = tf.random.uniform(shape=(4,1),minval=0, maxval=1, dtype=tf.int32)
        model_output = tf.random.uniform(shape=(4,1))

        loss = loss_function(target, model_output)
        # 
        loss_metric.update_state(values=loss)
        binary_accuracy_metric.update_state(target, model_output)
    tf.print(f"Epoch {e}: loss: {loss_metric.result()}")
    tf.print(f"Epoch {e}: accuracy: {binary_accuracy_metric.result()}")

    # RESETTING METRICS BEFORE EVALUATION
    loss_metric.reset_states()
    binary_accuracy_metric.reset_states()

    for val_batch in range(20):
        target = tf.random.uniform(shape=(4,1),minval=0, maxval=1, dtype=tf.int32)
        model_output = tf.random.uniform(shape=(4,1))

        loss = loss_function(target, model_output)
        loss_metric.update_state(values=loss)
        binary_accuracy_metric.update_state(target, model_output)

    tf.print(f"Epoch {e}: val_loss: {loss_metric.result()}")
    tf.print(f"Epoch {e}: val_accuracy: {binary_accuracy_metric.result()} \n")

# resetting the metric before the next epoch
loss_metric.reset_states()
binary_accuracy_metric.reset_states()

Epoch 0: loss: 1.0046213865280151
Epoch 0: accuracy: 0.5099999904632568
Epoch 0: val_loss: 0.9262159466743469
Epoch 0: val_accuracy: 0.5874999761581421 

Epoch 1: loss: 0.9797967076301575
Epoch 1: accuracy: 0.5229166746139526
Epoch 1: val_loss: 0.7920961380004883
Epoch 1: val_accuracy: 0.574999988079071 

Epoch 2: loss: 1.0067111253738403
Epoch 2: accuracy: 0.49166667461395264
Epoch 2: val_loss: 1.25363290309906
Epoch 2: val_accuracy: 0.4124999940395355 

Epoch 3: loss: 1.0154286623001099
Epoch 3: accuracy: 0.4937500059604645
Epoch 3: val_loss: 0.8198896646499634
Epoch 3: val_accuracy: 0.5375000238418579 

Epoch 4: loss: 1.0363298654556274
Epoch 4: accuracy: 0.49791666865348816
Epoch 4: val_loss: 1.026277780532837
Epoch 4: val_accuracy: 0.44999998807907104 



## Accuracy metrics for multi-class categorization tasks

In multi-class classification, we use the **CategoricalCrossentropy** as our loss function. To track the accuracy, we need to use a different keras metric, **CategoricalAccuracy**

In [28]:
# instantiate metric object (usually in the model's constructor, i.e. __init__, method)
loss_metric = tf.keras.metrics.Mean(name="loss")
accuracy_metric = tf.keras.metrics.CategoricalAccuracy(name="accuracy")
loss_function = tf.keras.losses.CategoricalCrossentropy()

for e in range(5):
    for train_batch in range(100):
        
        # create random one-hot targets (labels)
        target = tf.one_hot(tf.random.uniform(minval=0,maxval=9, shape=(4,),dtype=tf.int32), depth=10)

        # create random model output (for each element in the batch the values are probabilities that sum to 1)
        model_output = tf.nn.softmax(tf.random.uniform(shape=(4,10)), axis=-1)

        loss = loss_function(target, model_output)

        loss_metric.update_state(values=loss)
        accuracy_metric.update_state(target, model_output)
    tf.print(f"Epoch {e}: loss: {loss_metric.result()}")
    tf.print(f"Epoch {e}: accuracy: {accuracy_metric.result()}")

    # RESETTING METRICS BEFORE EVALUATION
    loss_metric.reset_states()
    accuracy_metric.reset_states()

    for val_batch in range(20):
        target = tf.one_hot(tf.random.uniform(minval=0,maxval=9, shape=(4,),dtype=tf.int32), depth=10)
        model_output = tf.nn.softmax(tf.random.uniform(shape=(4,10)), axis=-1)

        loss = loss_function(target, model_output)
        loss_metric.update_state(values=loss)
        accuracy_metric.update_state(target, model_output)

    tf.print(f"Epoch {e}: val_loss: {loss_metric.result()}")
    tf.print(f"Epoch {e}: val_accuracy: {accuracy_metric.result()} \n")

    # resetting the metric before the next epoch
    loss_metric.reset_states()
    accuracy_metric.reset_states()


Epoch 0: loss: 2.3252077102661133
Epoch 0: accuracy: 0.10000000149011612
Epoch 0: val_loss: 2.3609747886657715
Epoch 0: val_accuracy: 0.125 

Epoch 1: loss: 2.3456506729125977
Epoch 1: accuracy: 0.09749999642372131
Epoch 1: val_loss: 2.370378255844116
Epoch 1: val_accuracy: 0.0625 

Epoch 2: loss: 2.3308346271514893
Epoch 2: accuracy: 0.10750000178813934
Epoch 2: val_loss: 2.3247058391571045
Epoch 2: val_accuracy: 0.15000000596046448 

Epoch 3: loss: 2.308459997177124
Epoch 3: accuracy: 0.11249999701976776
Epoch 3: val_loss: 2.3280162811279297
Epoch 3: val_accuracy: 0.11249999701976776 

Epoch 4: loss: 2.346121311187744
Epoch 4: accuracy: 0.09749999642372131
Epoch 4: val_loss: 2.358342170715332
Epoch 4: val_accuracy: 0.0625 



## Conclusion

- We can use keras metrics instead of tedious numpy loss tracking and metric computations. 

- Keras metrics can be used with Tensorflow's graph mode, while list appends and numpy operations generally can not

- The convenient compile and fit methods of the tf.keras.Model class use keras metrics under the hood. 
    - Starting to use the metric objects gets us a step closer to being able to use these tools.



Next: **Tensorboard for logging**, log all possible kinds of training data to the tensorboard